In [ ]:
# default_exp process_bids

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
#export
import argparse
import os
import subprocess
import nibabel
import numpy
from glob import glob
from voxelwiseencoding.preprocessing import preprocess_bold_fmri, make_X_Y
from voxelwiseencoding.encoding import get_ridge_plus_scores, ridge_gridsearch_per_target
import json
import joblib
import numpy as np
from nilearn.masking import unmask
from nilearn.image import new_img_like, concat_imgs
from nilearn.masking import compute_epi_mask
from nibabel import save

/home/mboos/anaconda3/envs/mne/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/mboos/anaconda3/envs/mne/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


# Train voxel-wise encoding models on BIDS dataset
> Functions for extracting relevant BOLD and stimulus data and metadata of a BIDS dataset and for running voxel-wise encoding models.

In [ ]:
#export
#hide

def create_stim_filename_from_args(subject_label, **kwargs):
    '''Creates an expression corresponding to the stimulus files. It does not differentiate between json and tsv(.gz) files yet.'''
    stim_expr = ['sub-{}'.format(subject_label),
                 'ses-{}'.format(kwargs.get('ses')) if kwargs.get('ses') else None,
                 'task-{}'.format(kwargs.get('task')) if kwargs.get('task') else None,
                 'desc-{}'.format(kwargs.get('desc')) if kwargs.get('desc') else None,
                 '*',
                 'recording-{}'.format(kwargs.get('recording')) if kwargs.get('recording') else None,
                 'stim']
    stim_expr = '_'.join([term for term in stim_expr if term])
    # TODO: change hacky way to glob
    return stim_expr.replace('_*_', '_*')

def create_output_filename_from_args(subject_label, **kwargs):
    '''Creates filename for the model output'''
    output_expr = ['sub-{}'.format(subject_label),
                 'ses-{}'.format(kwargs.get('ses')) if kwargs.get('ses') else None,
                 'task-{}'.format(kwargs.get('task')) if kwargs.get('task') else None,
                 'desc-{}'.format(kwargs.get('desc')) if kwargs.get('desc') else None,
                 'recording-{}'.format(kwargs.get('recording')) if kwargs.get('recording') else None]
    output_expr = '_'.join([term for term in output_expr if term])
    return output_expr

#TODO: make globbable for different runs
def create_metadata_filename_from_args(subject_label, **kwargs):
    '''Creates filename for task metadata'''
    metadata_expr = ['sub-{}'.format(subject_label),
                     'task-{}'.format(kwargs.get('task')) if kwargs.get('task') else None,
                     'bold.json']
    metadata_expr = '_'.join([term for term in metadata_expr if term])
    return metadata_expr


def create_bold_glob_from_args(subject_label, **kwargs):
    '''Creates a globbable expression corresponding to the bold NifTIs to be used.'''
    bold_expr = ['sub-{}'.format(subject_label),
                 'ses-{}'.format(kwargs.get('ses')) if kwargs.get('ses') else None,
                 'task-{}'.format(kwargs.get('task')) if kwargs.get('task') else None,
                 'desc-{}'.format(kwargs.get('desc')) if kwargs.get('desc') else None,
                 '*_bold*.nii.gz']
    bold_expr = '_'.join([term for term in bold_expr if term])
    return bold_expr.replace('_*_', '_*')


def run(command, env={}):
    '''Runs the given command in local environment'''
    merged_env = os.environ
    merged_env.update(env)
    process = subprocess.Popen(command, stdout=subprocess.PIPE,
                               stderr=subprocess.STDOUT, shell=True,
                               env=merged_env)
    while True:
        line = process.stdout.readline()
        line = str(line, 'utf-8')[:-1]
        print(line)
        if line == '' and process.poll() is not None:
            break
    if process.returncode != 0:
        raise Exception("Non zero return code: {}".format(process.returncode))

In [ ]:
#export

def get_func_bold_directory(subject_label, bids_dir, **kwargs):
    '''Returns a path to the directory in which the bold files of the given subject reside

    Parameters

        subject_label : the BIDS subject label
        bids_dir : the path to the BIDS directory
        ses : session indicator, optional
        kwargs : additional arguments

    Returns
        bold_folder_name

    '''
    try:
        bold_folder = [bids_dir,
                       'sub-{}'.format(subject_label),
                       'ses-{}'.format(kwargs.get('ses')) if kwargs.get('ses') else None,
                       'func']
    except KeyError:
        raise ValueError('bids_dir argument is required.')
    bold_folder_name = os.path.join(*[term for term in bold_folder if term])
    # check if path exists, since func can be missing for derivatives
    if not os.path.exists(bold_folder_name):
        bold_folder_name = os.path.join(*[term for term in bold_folder[:-1] if term])
    return bold_folder_name

In [ ]:
#export

def process_bids_subject(subject_label, bids_dir, ses=None, task=None, desc=None, recording=None, **kwargs):
    '''Localizes BOLD files and stimulus files for subject_label in BIDS folder structure

    Parameters

        subject_label : the BIDS subject label
        bids_dir : the path to the BIDS directory
        ses : session indicator, optional
        task : task indicator, optional
        desc : description indicatr, optional
        recording : recording indicator, optional
        kwargs : additional arguments

    Returns
        tuple of (list of bold files, path to task meta data,
        list of stimulus tsv files, list of stimulus json files)
    '''
    bold_folder = get_func_bold_directory(subject_label, bids_dir,
                                          ses=ses, task=task, desc=desc,
                                          recording=recording, **kwargs)
    bold_glob = create_bold_glob_from_args(subject_label,
                                           ses=ses, task=task, desc=desc,
                                           recording=recording, **kwargs)
    bold_files = sorted(glob(os.path.join(bold_folder, bold_glob)))
    stim_glob = create_stim_filename_from_args(subject_label,
                                               ses=ses, task=task, desc=desc,
                                               recording=recording, **kwargs)

    # subject specific metadata takes precedence over other metadata
    try:
        with open(os.path.join(bold_folder,
                               create_metadata_filename_from_args(subject_label, task=task, **kwargs)), 'r') as fl:
            task_meta = json.load(fl)
    except FileNotFoundError:
        with open(os.path.join(bids_dir, 'task-{}_bold.json'.format(task)), 'r') as fl:
            task_meta = json.load(fl)

    # first check if subject specific stimulus files exist
    stim_tsv = glob(os.path.join(bold_folder, '.'.join([stim_glob, 'tsv.gz'])))
    if not stim_tsv:
        # try to get uncompressed tsv
        stim_tsv = glob(os.path.join(bold_folder, '.'.join([stim_glob, 'tsv'])))
        if not stim_tsv:
            # try to get tsvs in root directory without subject specifier
            root_glob = '_'.join(stim_glob.split('_')[1:])
            stim_tsv = glob(os.path.join(bids_dir,
                                            '.'.join([root_glob, 'tsv.gz'])))
            if not stim_tsv:
                # and check again in root for tsv
                stim_tsv = glob(os.path.join(bids_dir,
                                                '.'.join([root_glob, 'tsv'])))
                if not stim_tsv:
                    raise ValueError('No stimulus files found! [Mention naming scheme and location here]')
    stim_tsv = sorted(stim_tsv)
    stim_json = sorted(glob(os.path.join(bold_folder, '.'.join([stim_glob, 'json']))))
    if not stim_json:
        raise ValueError('No stimulus json files found!'
                         'These should be in the same folder as the functional data.')

    if not (len(stim_tsv) == len(stim_json) and len(stim_json) == len(bold_files)):
        raise ValueError('Number of stimulus tsv, stimulus json, and BOLD files differ.'
                ' Stimulus json: {} \n stimulus tsv: {} \n BOLD: {}'.format(stim_json, stim_tsv, bold_files))

    return bold_files, task_meta, stim_tsv, stim_json

In [ ]:
#export

def run_model_for_subject(subject_label, bids_dir, mask=None, bold_prep_kwargs=None,
                          preprocess_kwargs=None, encoding_kwargs=None, validate=True,
                          **kwargs):
    '''Runs voxel-wise encoding model for a single subject and returns Ridges and scores

    Parameters

        subject_label : the BIDS subject label
        bids_dir : the path to the BIDS directory
        mask : path to mask file or 'epi' if an epi mask should be computed from the first BOLD run
        bold_prep_kwargs : None or dict containing the parameters for preprocessing the BOLD files
                           everything that is accepted by nilearn's clean function is an acceptable parameter
        preprocess_kwargs : None or dict containing the parameters for lagging and aligning fMRI and stimulus
                            acceptable parameters are ones used by preprocessing.make_X_Y
        encoding_kwargs : None or dict containing the parameters for the encoding model
                          acceptable parameters are ones used by encoding.get_ridge_plus_scores
        validate : Boolean, default True,
                   whether to validate the model via cross-validation and return
                   a model for each train/test split and scores for each test set
        kwargs : additional BIDS specific arguments such as task, ses, desc, and recording

    Returns
        list of Ridge regressions, scores per voxel per fold

    '''
    if bold_prep_kwargs is None:
        bold_prep_kwargs = {}
    if encoding_kwargs is None:
        encoding_kwargs = {}
    if preprocess_kwargs is None:
        preprocess_kwargs = {}


    bold_files, task_meta, stim_tsv, stim_json = process_bids_subject(subject_label, bids_dir, **kwargs)

    # compute epi mask if required
    if mask == 'epi':
        mask = compute_epi_mask(bold_files[0])

    # do BOLD preprocessing
    preprocessed_data = []
    for bold_file in bold_files:
        preprocessed_data.append(preprocess_bold_fmri(bold_file, mask=mask, **bold_prep_kwargs))

    # load stimulus
    stim_meta = []
    stimuli = []
    for tsv_fl, json_fl in zip(stim_tsv, stim_json):
        with open(json_fl, 'r') as fl:
            stim_meta.append(json.load(fl))
        stimuli.append(np.loadtxt(tsv_fl, delimiter='\t'))

    start_times = [st_meta['StartTime'] for st_meta in stim_meta]
    stim_TR = 1. / stim_meta[0]['SamplingFrequency']

    # temporally align stimulus and fmri data
    stimuli, preprocessed_data = make_X_Y(
        stimuli, preprocessed_data, task_meta['RepetitionTime'],
        stim_TR, start_times=start_times, **preprocess_kwargs)
    
    if validate:
        # compute ridge and scores for folds
        ridges, scores = get_ridge_plus_scores(stimuli, preprocessed_data, **encoding_kwargs)
        return ridges, scores, mask
    else:
        ridges = ridge_gridsearch_per_target(stimuli, preprocessed_data, **encoding_kwargs)
        return ridges, None, mask

In [ ]:
show_doc(process_bids_subject)

<h4 id="process_bids_subject" class="doc_header"><code>process_bids_subject</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>process_bids_subject</code>(**`subject_label`**, **`bids_dir`**, **`ses`**=*`None`*, **`task`**=*`None`*, **`desc`**=*`None`*, **`recording`**=*`None`*, **\*\*`kwargs`**)

Localizes BOLD files and stimulus files for subject_label in BIDS folder structure

Parameters

    subject_label : the BIDS subject label
    bids_dir : the path to the BIDS directory
    ses : session indicator, optional
    task : task indicator, optional
    desc : description indicatr, optional
    recording : recording indicator, optional
    kwargs : additional arguments

Returns
    tuple of (list of bold files, path to task meta data,
    list of stimulus tsv files, list of stimulus json files)

In [ ]:
show_doc(get_func_bold_directory)

<h4 id="get_func_bold_directory" class="doc_header"><code>get_func_bold_directory</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>get_func_bold_directory</code>(**`subject_label`**, **`bids_dir`**, **\*\*`kwargs`**)

Returns a path to the directory in which the bold files of the given subject reside

Parameters

    subject_label : the BIDS subject label
    bids_dir : the path to the BIDS directory
    ses : session indicator, optional
    kwargs : additional arguments

Returns
    bold_folder_name